# Lesson 10 


First 15 minutes of the lecture are a showcase of student work and then a review / recap of how stable diffusion works. 



Then he talks about two paper that came out recently (relative to the lecture)
[Progressive Distillation for Fast Sampling of Diffusion Models](https://arxiv.org/abs/2202.00512) [paper walkthrough](https://www.youtube.com/watch?v=ZXuK6IRJlnk) .
This takes use from 60 steps of denoising to just a few using 'distillation' of the model (which is a form of transfer learning).

The teacher model is our fully trained stable diffusion model.  Then a student model (a unnet) is trained to skip over steps prgressively.  

Another paper from same group shows how to use distillation for guided diffusion models:
[On Distillation of Guided Diffusion Models](https://arxiv.org/abs/2210.03142)


At about 26 minutes:

 [Imagic: Text-Based Real Image Editing with Diffusion Models](https://arxiv.org/abs/2210.09276)